(file-types:notebooks)=
# La réforme des retraites à l'Assemblée Nationale

## Introduction
Si le gouvernement a utilisé l'article 49.3, la loi a toute fois était débattu en première lecture à l'assemblée nationale du 6 février au 17 février. Depuis 2017, il est possible d'avoir accès sur le site de l'assemblée national au retranscription des séances publiques de l'Assemblée Nationale. A ce jour, il n'existe pas de solutions pour télécharger ces transcriptions dans un (ou plusieurs) fichiers de données (csv, xlsx, etc...). Une des possibilités est d'utiliser le *web-scrapping* qui consiste à extraire des informations directement du code HTML d'une page web. Ici, ces pages webs correspondront aux transcriptions de chaque séances. Le script Python pour scrapper le site de l'assemblée nationale est disponible [ici](https://github.com/Jacobe2169/debat_parlement_text_analysis/blob/master/1_get_transcript_seance_assemblee.py).


## Chargement des données

Pour commencer, nous chargeons plusieurs modules/librairies permettant de charger les données et produire les différentes visualisations.

In [1]:
import pandas as pd
import plotly.express as px
from lib.figures import *
from lib.constant import *
from lib.utils import *
from bokeh.io import output_notebook
output_notebook(hide_banner=True)

Une fois le chargement des modules effectués, nous chargons les données dans la *dataframe* (ou table de données) `parlement_df`.

In [2]:


parlement_df = pd.read_parquet('data/parlement_retraite_data.parquet')
parlement_df["is_keywords_label"] = parlement_df.is_keywords.apply(lambda x: "Oui" if x else "Non")
parlement_df.head()

,seance_number,full_text,section,subsection,date,username,groupe_sigle,len_intervention,lemmatization,keywords_detected,is_keywords,is_keywords_label
0,1,"La parole est à M. le ministre du travail, du ...",Projet de loi de financement rectificative de...,Présentation,2023-02-06,GOV,GOV,185,"le parole être à m. le ministre de travail , d...","[travail, plein emploi, commission de affaire ...",True,Oui
1,1,"Avec tout le respect que je vous porte, madame...",Projet de loi de financement rectificative de...,Présentation,2023-02-06,valerie-rabault,SER,277,"avec tout le respect que je vous porte , monsi...","[article 122, motion référendaire, vote, ratio...",True,Oui
2,1,"Le pays tout entier a dit, sur tous les tons, ...",Projet de loi de financement rectificative de...,Présentation,2023-02-06,raquel-garrido,LFI,375,"le pays tout entier avoir dire , sur tout le t...","[article 122, motion référendaire, pays, retra...",True,Oui
3,1,"Contrairement à tout usage, et devant le silen...",Projet de loi de financement rectificative de...,Présentation,2023-02-06,sebastien-jumel,GDR,248,"contrairement à tout usage , et devant le sile...","[article 122, motion référendaire, effet, réfo...",True,Oui
4,1,"Le débat qui nous occupe, et qui pourrait abou...",Projet de loi de financement rectificative de...,Présentation,2023-02-06,benjamin-lucas,ECO,339,"le débat qui nous occupe , et qui pouvoir abou...","[motion référendaire, pays, démocratie, vie en...",True,Oui


In [3]:
average_number_of_words_per_intervention(parlement_df)

In [4]:
intervention_frequency_per_group(parlement_df)

## Analyse des mots-clés

### Quels mots-clés ?
Les mots clés sont identifiés parmi différentes extractions automatiques. La première est en utilisant l'outil Biotex[^biotex-article] qui extrait des mots-clés à l'aide de motifs syntaxiques identifiées dans des corpus. La seconde correspond à une extraction de n-grammes, ou séquences de mots, qui se répètent souvent.


[^biotex-article]: Juan Antonio Lossio-Ventura, Clement Jonquet, Mathieu Roche, Maguelonne Teisseire. BIOTEX: A system for Biomedical Terminology Extraction, Ranking, and Validation. ISWC: International Semantic Web Conference, Oct 2014, Riva del Garda, Italy. pp.157-160. [Version Python](https://github.com/Jacobe2169/Biotex4Py)

L'ensemble des mots-clés sont listés dans la table ci-dessous.

In [5]:
keyword_df = pd.read_excel("data/keywords_selected.ods",index_col=0)[["term"]]
datatable_from_pandas(keyword_df)

### Proportion des interventions contenant des mots-clés 

In [6]:
fig_parl = px.histogram(parlement_df,
                        x="groupe_sigle",
                        pattern_shape="is_keywords_label",
                        labels={"count":"","groupe_sigle":"Groupe Politique","is_keywords_label":"Présence de mots-clés"},
                        color_discrete_map=gp_politique_color, color="groupe_sigle")
fig_parl.update_layout(yaxis_title="# Interventions")

### Distribution des mots-clés par groupe parlementaire

In [7]:
from bokeh.models import TabPanel, Tabs

df = getCountDataframe(parlement_df,top_n=10)

tab1 = TabPanel(child=occurrenceDistributionPerGroupePolitique(df[df.num_words == 1]), title="1 mot")
tab2 = TabPanel(child=occurrenceDistributionPerGroupePolitique(df[df.num_words == 2]), title="2 mots")
tab3 = TabPanel(child=occurrenceDistributionPerGroupePolitique(df[df.num_words == 3]), title="3 mots")

show(Tabs(tabs=[tab1, tab2,tab3],sizing_mode ="stretch_width"))

### Distribution des mots-clés par orientation politique

In [8]:
pd.DataFrame(list(droite_ou_gauche_centre.items()),columns="Groupe Parlementaire,Orientation".split(","))

,Groupe Parlementaire,Orientation
0,GOV,centre
1,SER,gauche
2,LFI,gauche
3,GDR,gauche
4,ECO,gauche
5,NI,aucun
6,UDI,droite
7,LREM,centre
8,MODEM,centre
9,LIOT,droite


In [10]:
from bokeh.models import TabPanel, Tabs

df = getCountDataframe(parlement_df,top_n=10)

tab1 = TabPanel(child=occurrenceDistributionPerPolitiqueOrientation(df[df.num_words == 1]), title="1 mot")
tab2 = TabPanel(child=occurrenceDistributionPerPolitiqueOrientation(df[df.num_words == 2]), title="2 mots")
tab3 = TabPanel(child=occurrenceDistributionPerPolitiqueOrientation(df[df.num_words == 3]), title="3 mots")

show(Tabs(tabs=[tab1, tab2,tab3],sizing_mode ="stretch_width"))

c:\Users\jacqu\Dropbox\Projets\2023\debat_retraite_book\lib\figures.py:233: UserWarning:


You are attempting to set `plot.legend.click_policy` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.


c:\Users\jacqu\Dropbox\Projets\2023\debat_retraite_book\lib\figures.py:233: UserWarning:


You are attempting to set `plot.legend.click_policy` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.


c:\Users\jacqu\Dropbox\Projets\2023\debat_retraite_book\lib\figures.py:233: UserWarning:


You are attempting to set `plot.legend.click_policy` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter se